In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess dataset
image_dir = Path(r'E:\Abroad period research\new idea implementation codes\Second part of the paper\26 features results\small dataset for hyperparameter tunning')
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

# Convert file paths and labels to a DataFrame
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1).sample(frac=1).reset_index(drop=True)

# Split data into train, validation, and test sets
train_df, temp_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)
val_df, test_df = train_test_split(temp_df, train_size=0.5, shuffle=True, random_state=1)

# Create data generators
def create_data_generators(train_df, val_df, test_df, batch_size):
    train_gen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
        shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
    )
    val_test_gen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_gen.flow_from_dataframe(train_df, x_col='Filepath', y_col='Label',
                                                 target_size=(224, 224), color_mode='rgb', class_mode='categorical',
                                                 batch_size=batch_size, shuffle=True, seed=0)
    val_images = val_test_gen.flow_from_dataframe(val_df, x_col='Filepath', y_col='Label',
                                                  target_size=(224, 224), color_mode='rgb', class_mode='categorical',
                                                  batch_size=batch_size, shuffle=False)
    test_images = val_test_gen.flow_from_dataframe(test_df, x_col='Filepath', y_col='Label',
                                                   target_size=(224, 224), color_mode='rgb', class_mode='categorical',
                                                   batch_size=batch_size, shuffle=False)
    return train_images, val_images, test_images

# Model-building function
def build_model(learning_rate=0.001):
    pretrained_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, pooling='avg', weights='imagenet')
    pretrained_model.trainable = True
    for layer in pretrained_model.layers[:-50]:  # Fine-tune last 50 layers
        layer.trainable = False

    model = tf.keras.Sequential([
        pretrained_model,
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(len(train_df['Label'].unique()), activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.005, 0.01, 0.02],
    'epochs': [10, 30, 50],
    'batch_size': [16, 32, 64, 128]
}

# Grid search
best_accuracy = 0
best_params = None
train_images, val_images, test_images = create_data_generators(train_df, val_df, test_df, batch_size=32)

for lr in param_grid['learning_rate']:
    for epochs in param_grid['epochs']:
        for batch_size in param_grid['batch_size']:
            print(f"Training with lr={lr}, epochs={epochs}, batch_size={batch_size}")
            model = build_model(learning_rate=lr)
            history = model.fit(train_images, epochs=epochs, validation_data=val_images, batch_size=batch_size, verbose=0)
            
            # Evaluate on validation set
            val_loss, val_accuracy = model.evaluate(val_images, verbose=0)
            print(f"Validation accuracy: {val_accuracy:.4f}")

            # Track best parameters
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'learning_rate': lr, 'epochs': epochs, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_params}")
print(f"Best Validation Accuracy: {best_accuracy:.4f}")

# Final evaluation on test set
best_model = build_model(learning_rate=best_params['learning_rate'])
best_model.fit(train_images, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0)
test_loss, test_acc = best_model.evaluate(test_images, verbose=0)
print(f"Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}")


Found 280 validated image filenames belonging to 4 classes.
Found 60 validated image filenames belonging to 4 classes.
Found 60 validated image filenames belonging to 4 classes.
Training with lr=0.001, epochs=10, batch_size=16


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Validation accuracy: 0.3000
Training with lr=0.001, epochs=10, batch_size=32
Validation accuracy: 0.4500
Training with lr=0.001, epochs=10, batch_size=64
Validation accuracy: 0.2333
Training with lr=0.001, epochs=10, batch_size=128
Validation accuracy: 0.3000
Training with lr=0.001, epochs=30, batch_size=16
Validation accuracy: 0.2500
Training with lr=0.001, epochs=30, batch_size=32
Validation accuracy: 0.4167
Training with lr=0.001, epochs=30, batch_size=64
Validation accuracy: 0.3833
Training with lr=0.001, epochs=30, batch_size=128
Validation accuracy: 0.4333
Training with lr=0.001, epochs=50, batch_size=16
Validation accuracy: 0.5000
Training with lr=0.001, epochs=50, batch_size=32
Validation accuracy: 0.4333
Training with lr=0.001, epochs=50, batch_size=64
Validation accuracy: 0.3667
Training with lr=0.001, epochs=50, batch_size=128
Validation accuracy: 0.5833
Training with lr=0.005, epochs=10, batch_size=16
Validation accuracy: 0.4833
Training with lr=0.005, epochs=10, batch_size